In [0]:
from pyspark.sql.functions import *


In [0]:
df_gold01 = spark.read.format("delta").load("/Volumes/workspace/default/gold/corte_agua/cortes_por_bairro/")
df_gold03 = spark.read.format("delta").load("/Volumes/workspace/default/gold/corte_agua/cortes_por_mes/")
df_gold04 = spark.read.format("delta").load("/Volumes/workspace/default/gold/corte_agua/gold_06/")
df_gold05 = spark.read.format("delta").load("/Volumes/workspace/default/gold/corte_agua/gold_07/")
df_gold06 = spark.read.format("delta").load("/Volumes/workspace/default/gold/corte_agua/ranking_top_bairros/")



In [0]:
display(df_gold01)
display(df_gold03)
display(df_gold04)
display(df_gold05)
display(df_gold06)


In [0]:
display(df_gold02.orderBy("qtd_cortes", ascending=False))


In [0]:
import plotly.express as px


df_top10 = spark.read.format("delta").load(
    "/Volumes/workspace/default/gold/corte_agua/top10_bairros/"
)
pdf_top10 = df_top10.toPandas()

fig = px.bar(
    pdf_top10.sort_values("qtd_cortes"),
    x="qtd_cortes",   # valores
    y="bairro",       # nomes dos bairros
    orientation="h",
    color="qtd_cortes",
    text="qtd_cortes",
    title="Top 10 Bairros com Mais Cortes em todos os anos",
    color_continuous_scale="reds"
)
fig.update_traces(textposition="outside")
fig.update_layout(template="plotly_white")
fig.show()



In [0]:
import plotly.express as px


df_status_ano = spark.read.format("delta").load(
    "/Volumes/workspace/default/gold/corte_agua/gold_06//"
)
pdf_status_ano = df_status_ano.toPandas()


fig = px.bar(
    pdf_status_ano,
    x="ano_corte",         
    y="total_cortes",      
    color="status_do_corte",   # agrupamento por status
    barmode="group",
    text="total_cortes",
    title="Cortes Efetuados vs Pendentes por Ano",
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_traces(textposition="outside")
fig.update_layout(template="plotly_white")
fig.show()


In [0]:
import plotly.express as px


df_mes = spark.read.format("delta").load(
    "/Volumes/workspace/default/gold/corte_agua/cortes_por_mes/"
)
pdf_mes = df_mes.toPandas()

# Mapear números para nomes dos meses
meses = {1:"Janeiro",2:"Fevereiro",3:"Março",4:"Abril",5:"Maio",6:"Junho",
         7:"Julho",8:"Agosto",9:"Setembro",10:"Outubro",11:"Novembro",12:"Dezembro"}
pdf_mes['mes_nome'] = pdf_mes['mes'].map(meses)


def top5_por_ano(df, ano_col='ano', qtd_col='qtd_cortes'):
    return df.groupby(ano_col, group_keys=False).apply(lambda x: x.nlargest(5, qtd_col))

pdf_top5 = top5_por_ano(pdf_mes)

# Criar uma pizza por ano
anos = pdf_top5['ano'].unique()
for ano in anos:
    pdf_ano = pdf_top5[pdf_top5['ano'] == ano]
    fig = px.pie(
        pdf_ano,
        names='mes_nome',
        values='qtd_cortes',
        title=f"Top 5 Meses com Mais Cortes - Ano {ano}",
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    fig.update_traces(textposition='inside', textinfo='value+label')
    fig.update_layout(template="plotly_white")
    fig.show()
